In [ ]:
!pip install pytube
!pip install whisper
!pip install -U scikit-learn
!pip install scikit-learn==1.2.2
!pip install --upgrade --no-cache-dir git+https://github.com/openai/whisper.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=b6db7938e586964d6d64a8608702cb3d6cf40a050787fdae0752db6977bc38eb
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1


In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import subprocess
import os
import whisper
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pytube import YouTube


In [ ]:

# YouTube API anahtarınızı buraya ekleyin
API_KEY = ''

def search_youtube(query):
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    request = youtube.search().list(q=query, part='snippet', type='video', maxResults=1)
    response = request.execute()
    video_id = response['items'][0]['id']['videoId']
    video_title = response['items'][0]['snippet']['title']
    return video_id, video_title

# Kullanıcıdan şarkı adını al
song_name = input("Müzik önerisi için bir şarkı adı giriniz: ")

# YouTube'dan video ID'sini ve başlığını al
video_id, video_title = search_youtube(song_name)
print(f"Video ID bulundu: {video_id} - Başlık: {video_title}")


Müzik önerisi için bir şarkı adı giriniz: do you like me
Video ID bulundu: X8-t1c5Kd4M - Başlık: Do You Like Me?


In [ ]:

def download_video(video_id):
    try:
        yt = YouTube(f'https://www.youtube.com/watch?v={video_id}')
        stream = yt.streams.filter(only_audio=True).first()
        stream.download(output_path='./', filename='video.mp4')
        return True
    except Exception as e:
        print(f"Hata oluştu: {e}")
        return False

def extract_transcript(video_file):
    try:
        # Whisper ile transkript alma
        whisper_command = f'python whisper/whisper.py --input_file {video_file} --output_file output.srt'
        subprocess.run(whisper_command, shell=True, check=True)
        with open('output.srt', 'r', encoding='utf-8') as file:
            transcript = file.read()
        return transcript
    except Exception as e:
        print(f"Hata oluştu: {e}")
        return None

# Videoyu indir
if download_video(video_id):
    print("Video başarıyla indirildi.")

    # Video dosyasını Whisper ile işle
    video_file = './video.mp4'
    transcript = extract_transcript(video_file)

    if transcript:
        print("Video metni başarıyla alındı:")
        print(transcript)
    else:
        print("Video metni alınamadı.")
else:
    print("Video indirilemedi.")


Hata oluştu: get_throttling_function_name: could not find match for multiple
Video indirilemedi.


In [ ]:



def get_video_transcript(video_path, output_txt_path):
    """
    Verilen video dosyasından metni Whisper ile alır ve belirtilen TXT dosyasına kaydeder.

    :param video_path: Video dosyasının yolu (MP4 formatı)
    :param output_txt_path: Metnin kaydedileceği TXT dosyasının yolu
    :return: None
    """
    # Whisper modelini yükle
    model = whisper.load_model("base") # Ensure you are using the latest version of Whisper

    # Video dosyasını transkribe et
    result = model.transcribe(video_path)

    # Transkripti TXT dosyasına kaydet
    with open(output_txt_path, "w", encoding="utf-8") as file:
        file.write(result["text"])

    print(f"Video transkripti başarıyla kaydedildi: {output_txt_path}")

# Örnek kullanım
video_path = "/content/video.mp4"
output_txt_path = "/content/müzik.txt"

get_video_transcript(video_path, output_txt_path)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 88.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


RuntimeError: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --ena

In [ ]:


# Gerekli NLTK paketlerini indir
nltk.download('punkt')
nltk.download('stopwords')

# Metin dosyasını oku
with open("/content/müzik.txt", "r", encoding="utf-8") as file:
    lyrics = file.read()

print(lyrics)


In [ ]:
def extract_keywords(text, num_keywords=10):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    freq_dist = nltk.FreqDist(filtered_tokens)
    keywords = [word for word, freq in freq_dist.most_common(num_keywords)]

    return keywords

# Anahtar kelimeleri belirle
keywords = extract_keywords(lyrics)
print("Anahtar Kelimeler:", keywords)


In [ ]:

# YouTube Data API anahtarınız
API_KEY = ''

def get_youtube_recommendations(keywords, api_key, max_results=10):
    youtube = build('youtube', 'v3', developerKey=api_key)
    recommendations = []

    for keyword in keywords:
        request = youtube.search().list(
            part='snippet',
            q=keyword,
            maxResults=max_results,
            type='video',
            videoCategoryId='10',  # Music category
        )
        response = request.execute()

        for item in response['items']:
            video_info = {
                'title': item['snippet']['title'],
                'channel': item['snippet']['channelTitle'],
                'url': f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            recommendations.append(video_info)

    return recommendations

# YouTube API üzerinden önerilen şarkıları çek
youtube_recommendations = get_youtube_recommendations(keywords, API_KEY)

# Önerilen şarkıları kullanıcıya göster
print("Önerilen şarkılar:")
for song in youtube_recommendations:
    print(f"Başlık: {song['title']}, Kanal: {song['channel']}, URL: {song['url']}")

